# Working with data via MXNet NDArray

MXNet's NDArray is a tensor data structure similar to numpy's multi-dimensional array. 
It confers a few key advantages. First, it supports asynchronous computation on CPU and GPU, and distributed cloud architectures. Second, it provides support for automatic differentiation. These properties make it especially suited as a library for both deep learning research and production.


## Getting started

First, let's import mxnet and (for convenience) mxnet.ndarray, the only dependencies we'll need in this tutorial.

In [2]:
import mxnet as mx
import mxnet.ndarray as nd

First, we'll create a 6x4 matrix, without initializing values

In [3]:
x = nd.empty(shape=(6,4))
print(x)

[[  3.52694761e-37   4.56991455e-41   5.47407716e-37   0.00000000e+00]
 [ -2.27570357e-17   4.56977442e-41   8.40779079e-45   4.56977442e-41]
 [  1.42932443e-43   1.26116862e-44   1.51340234e-43   1.41531145e-43]
 [  1.68155816e-44   1.55544130e-43   6.86636248e-44   2.10194770e-44]
 [  5.76533918e-37   0.00000000e+00   2.52233724e-44   4.56991455e-41]
 [  1.38728548e-43   2.94272678e-44   1.55544130e-43   1.59748025e-43]]
<NDArray 6x4 @cpu(0)>


We can also create a 6x4 matrix, initialized with values drawn from a standard normal distribution.

In [4]:
x = nd.random_normal(shape=(6,4))
print(x)

[[ 2.21220636  1.16307867  0.7740038   0.48380461]
 [ 1.04344046  0.29956347  1.18392551  0.15302546]
 [ 1.89171135 -1.16881478 -1.23474145  1.55807114]
 [-1.771029   -0.54594457 -0.45138445 -2.35562968]
 [ 0.57938355  0.54144019 -1.85608196  2.67850661]
 [-1.9768796   1.25463438 -0.20801921 -0.54877394]]
<NDArray 6x4 @cpu(0)>


As in NumPy, we can get its shape

In [5]:
print(x.shape)

(6, 4)


We can also query it's size, which is equal to the product of the components of the shape.

In [6]:
print(x.size)

24


## Operations

There are multiple syntaxes for operations. Let’s see addition as an example

In [7]:
y = nd.random_normal(shape=(6,4))
c = x + y
print(c)

[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


In the previous example, MXNet allocates new memory for the sum ``x+y`` and assigns a reference to the variable ``c``. To make better use of memory, we often prefer to perform operations in place, reusing already allocated memory. 

In MXNet, we can perform inplace operations by assigning the results of operations using slice notation (``result[:] = ...``).

In [8]:
result = nd.zeros(shape=(6,4))
result[:] = x+y
print(result)

[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


## Slicing

MXNet NDArrays currently support slicing along the first (0th) axis only. To use arbitrary slicing patterns, you can convert NDArrays to numpy arrays and back.
#### comment 
*why saying it just supports slicing along the first axix while we can use "nd.slice_axis(result,axis=1, begin=0, end=2)", maybe I did not get your point here.*

In [9]:
result[2:4]

[[ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]]
<NDArray 2x4 @cpu(0)>

## Converting from MXNet NDArray to NumPy Array

Converting MXNet NDArrays to and from NumPy is easy. Note that the converted arrays do not share memory.

In [10]:
a = nd.ones(shape=(5))
print(a)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [11]:
b = a.asnumpy()
print(b)

[ 1.  1.  1.  1.  1.]


In [12]:
b[0] = 2
print(b)
print(a)

[ 2.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Converting from NumPy Array to MXNet NDArray

Constructing an MXNet NDarray from a NumPy Array is straightforward.

In [13]:
c = nd.array(b)
print(c)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Managing context

In MXNet, every array has a context. A context could be the CPU, or one of many GPUs. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. 


In [22]:
d = nd.array(b, mx.cpu()) # or .gpu() for GPU allocation.

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [23]:
e = d.copyto(mx.cpu(1)) # or copy to a GPU if any exists.
print(e)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(1)>


Often we only want to make a copy if the variable isn't already on the desired context. In these cases, we can call ``as_in_context()`` which only makes a copy when the source and target contexts are different.

In [24]:
f = d.as_in_context(mx.cpu(0))
print(f)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [25]:
print(d)
f[0] = 1 
print(d)

[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>
[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>
